In [2]:
import pandas as pd
data1 = pd.read_csv('data/Youtube01.csv')
data2 = pd.read_csv('data/Youtube02.csv')
data3 = pd.read_csv('data/Youtube03.csv')
df = pd.concat([data1, data2, data3])

In [3]:
df = df.dropna().rename(columns={"CONTENT": "text", "CLASS": "label"})

In [4]:
df.head()

,COMMENT_ID,AUTHOR,DATE,text,label
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [7]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [8]:
dataset = dataset.train_test_split(test_size=0.2)

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)
dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map: 100%|██████████| 228/228 [00:00<00:00, 23527.56 examples/s]


In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [12]:
training_args = TrainingArguments(
    output_dir="./spam_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\aiman\AppData\Local\Temp\ipykernel_12732\3161674938.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.175813,0.929825,0.927273,0.927273,0.927273
2,No log,0.151023,0.942982,0.940092,0.953271,0.927273
3,No log,0.158809,0.960526,0.960000,0.939130,0.981818
4,No log,0.150495,0.960526,0.958904,0.963303,0.954545
5,No log,0.149195,0.951754,0.950673,0.938053,0.963636


TrainOutput(global_step=285, training_loss=0.09924156456662897, metrics={'train_runtime': 19.0038, 'train_samples_per_second': 239.426, 'train_steps_per_second': 14.997, 'total_flos': 150681665971200.0, 'train_loss': 0.09924156456662897, 'epoch': 5.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.14919504523277283,
 'eval_accuracy': 0.9517543859649122,
 'eval_f1': 0.9506726457399103,
 'eval_precision': 0.9380530973451328,
 'eval_recall': 0.9636363636363636,
 'eval_runtime': 0.5809,
 'eval_samples_per_second': 392.498,
 'eval_steps_per_second': 6.886,
 'epoch': 5.0}

In [16]:
trainer.save_model("model/spam_detector_transformer")
tokenizer.save_pretrained("model/spam_detector_transformer")

('model/spam_detector_transformer\\tokenizer_config.json',
 'model/spam_detector_transformer\\special_tokens_map.json',
 'model/spam_detector_transformer\\vocab.txt',
 'model/spam_detector_transformer\\added_tokens.json',
 'model/spam_detector_transformer\\tokenizer.json')

In [17]:
model_dir = "model/spam_detector_transformer"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [22]:
import torch

def predict_spam(comment_texts):
    inputs = tokenizer(comment_texts, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    return predictions.tolist()

In [38]:
test_comments = [
    "Subscribe to my channel and win a free iPhone!",
    "What a beautiful performance! I cried 😭",
    "Click this link to get 1000 followers fast!",
    "Are you out of money? Press this link now",
]

predictions = predict_spam(test_comments)

for comment, pred in zip(test_comments, predictions):
    label = "SPAM" if pred == 1 else "NOT SPAM"
    print(f"{label}: {comment}")

SPAM: Subscribe to my channel and win a free iPhone!
NOT SPAM: What a beautiful performance! I cried 😭
SPAM: Click this link to get 1000 followers fast!
SPAM: Are you out of money? Press this link now
